In [1]:
from libraries import *
from parameters import *
import statsmodels.api as sm
import sklearn.feature_selection
import statsmodels.api as sm
import sklearn.feature_selection
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')
from numpy.random import seed
from numpy.random import randint
import statsmodels.formula.api as smf

import statsmodels.api as sm
%load_ext rpy2.ipython

import sys

In [2]:
os.getcwd()
os.chdir(projectDir)

In [3]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
adataC = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_08122020_PerGENE.h5ad')

In [5]:
adataC.obs

,barcode_rank,channel,empty_drops_FDR,log10_n_genes,log10_n_umis,mt_frac,n_counts,n_genes,n_umis,neg_log10_empty_drops_FDR,...,GENE_Zfp598_,GENE_Zfp651_,GENE_Zfp91_,GENE_Zmiz1_,GENE_Zmiz2_,GENE_Znrf2_,GENE_Znrf3_,GENE_Znrf4_,GENE_Zpbp2_,GENE_CONTROL_
AAACCCAGTGACACGA-1-E3-1,7600.5,1,0.0,3.226342,3.689398,0.082396,4891.0,1414,4891.0,4.0,...,0,0,0,0,0,0,0,0,0,0
AAACGAAAGCAGTCTT-1-E3-1,6446.0,1,0.0,3.285107,3.714833,0.059391,5186.0,1656,5186.0,4.0,...,0,0,0,0,0,0,0,0,0,0
AAACGAAAGCATTTGC-1-E3-1,8367.5,1,0.0,3.252853,3.673850,0.056792,4719.0,1572,4719.0,4.0,...,0,0,0,0,0,0,0,0,0,0
AAACGAATCTCAACCC-1-E3-1,4937.0,1,0.0,3.305996,3.756103,0.063300,5703.0,1721,5703.0,4.0,...,0,0,0,0,0,0,0,0,0,0
AAACGAATCTGGTGCG-1-E3-1,430.5,1,0.0,3.410609,3.969742,0.070012,9327.0,2158,9327.0,4.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGTTTCTGCTTTA-1-E4-24,5728.0,24,0.0,3.493319,4.010978,0.061330,10256.0,2611,10256.0,4.0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGAGTCATCCA-1-E4-24,3210.5,24,0.0,3.481443,4.091421,0.069108,12343.0,2548,12343.0,4.0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGAGTTAACAG-1-E4-24,13565.0,24,0.0,3.367356,3.846337,0.056980,7020.0,1990,7020.0,4.0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGCAGGACATG-1-E4-24,21270.0,24,0.0,3.060320,3.423246,0.040377,2650.0,1015,2650.0,4.0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
selKOs = pd.read_csv("EnrichedDepletedGuides.csv", index_col=0)
selKOs = list("GENE_" + selKOs.x + "_")
selKOs.append("GENE_CONTROL_")


In [7]:
myAdata = adataC[adataC.obs[selKOs].sum(axis=1) == 1]
myAdata.shape

(59998, 6685)

In [8]:
myAdata.obs[selKOs].sum(axis=0)

GENE_Vhl_          269
GENE_Rrp9_         340
GENE_Cul3_         252
GENE_Naca_         114
GENE_Paf1_          78
                 ...  
GENE_Pten_         232
GENE_Ambra1_       348
GENE_Trp53_        544
GENE_Nf1_          489
GENE_CONTROL_    41123
Length: 80, dtype: int64

In [9]:
myAdata.obs["subCellType"] = "DC2"
# myAdata.obs.loc[myAdata.obs.leiden.isin(['3']), "subCellType"] = "MacDC"
# myAdata.obs.loc[myAdata.obs.leiden.isin(['8']), "subCellType"] = "DC1"
# myAdata.obs.loc[myAdata.obs.leiden.isin(['5']), "subCellType"] = "MReg"
myAdata.obs.loc[myAdata.obs.leiden.isin(['3']), "subCellType"] = "Other"
myAdata.obs.loc[myAdata.obs.leiden.isin(['8']), "subCellType"] = "Other"
myAdata.obs.loc[myAdata.obs.leiden.isin(['5']), "subCellType"] = "Other"

In [10]:
guideMatrix = myAdata.obs[selKOs]
guideMatrix.shape

(59998, 80)

In [11]:
df = myAdata.obs[["n_genes", "mt_frac", "subCellType" ]]

guideMatrix = guideMatrix.join(df)
guideMatrix.shape

(59998, 83)

In [12]:
guideMatrix

,GENE_Vhl_,GENE_Rrp9_,GENE_Cul3_,GENE_Naca_,GENE_Paf1_,GENE_Rnf8_,GENE_Wdr3_,GENE_Rack1_,GENE_Myc_,GENE_Wdr74_,...,GENE_Zmiz2_,GENE_Cul5_,GENE_Pten_,GENE_Ambra1_,GENE_Trp53_,GENE_Nf1_,GENE_CONTROL_,n_genes,mt_frac,subCellType
AAACGAATCTCAACCC-1-E3-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1721,0.063300,DC2
AAACGCTAGTTGTAGA-1-E3-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,951,0.086056,DC2
AAACGCTCAGTGTACT-1-E3-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1518,0.108019,DC2
AAAGAACAGCGAGTCA-1-E3-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1300,0.051720,DC2
AAAGGATTCAGACAAA-1-E3-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1247,0.071198,DC2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGGAGTCCGTAGGC-1-E4-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,2252,0.074738,DC2
TTTGGTTAGTTTAGGA-1-E4-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1660,0.054039,DC2
TTTGGTTCACCTGATA-1-E4-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2023,0.082165,DC2
TTTGGTTGTCATCACA-1-E4-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1725,0.050018,DC2


In [13]:
guideMatrix.drop('GENE_CONTROL_', inplace=True, axis=1)

In [14]:
myAdata.X.shape

(59998, 6685)

In [15]:
expressionMatrix = pd.DataFrame(myAdata.X)

In [16]:
expressionMatrix.columns = myAdata.var_names

In [17]:
expressionMatrix

,Mrpl15,Lypla1,Tcea1,Atp6v1h,Rb1cc1,Pcmtd1,Rrs1,Vcpip1,Sgk3,Snhg6,...,Cacul1,Eif3a,Fam45a,Prdx3,Grk5,Zfp950,Csf2ra,Vamp7,AC149090.1,CAAA01118383.1
0,1.691574,1.675400,1.080451,-0.564251,-0.537373,-0.4147,1.501153,-0.515470,-0.486193,-0.286964,...,-0.505382,0.225831,-0.349419,3.306102,-0.29067,-0.271988,1.342306,-0.354616,2.460229,-0.399062
1,2.980780,-0.490747,3.086521,2.513951,-0.537373,-0.4147,2.700778,-0.515470,-0.486193,-0.286964,...,-0.505382,0.377125,-0.349419,-0.417314,-0.29067,-0.271988,1.069392,-0.354616,-0.517731,-0.399062
2,-0.489387,-0.490747,-0.656056,-0.564251,2.886541,-0.4147,-0.528264,1.982912,-0.486193,-0.286964,...,2.070611,1.589570,-0.349419,3.980204,-0.29067,-0.271988,0.570324,-0.354616,-0.517731,-0.399062
3,-0.489387,-0.490747,1.527742,1.868684,1.933220,-0.4147,-0.528264,-0.515470,-0.486193,-0.286964,...,-0.505382,0.624197,-0.349419,-0.417314,-0.29067,-0.271988,1.326897,-0.354616,-0.517731,2.769331
4,-0.489387,-0.490747,1.687905,2.047119,-0.537373,-0.4147,-0.528264,-0.515470,-0.486193,-0.286964,...,-0.505382,0.762567,-0.349419,-0.417314,-0.29067,-0.271988,-1.103932,-0.354616,-0.517731,-0.399062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59993,0.992061,-0.490747,-0.656056,-0.564251,-0.537373,-0.4147,0.850245,0.869044,-0.486193,-0.286964,...,-0.505382,0.758629,-0.349419,1.296598,-0.29067,2.145239,-0.176119,-0.354616,-0.517731,1.312299
59994,-0.489387,-0.490747,-0.656056,-0.564251,-0.537373,-0.4147,-0.528264,-0.515470,-0.486193,-0.286964,...,-0.505382,1.273016,-0.349419,2.213726,3.17844,-0.271988,-1.103932,2.574493,-0.517731,-0.399062
59995,-0.489387,1.265500,0.751852,1.837076,1.901123,-0.4147,1.117126,2.014502,-0.486193,-0.286964,...,-0.505382,0.599478,-0.349419,-0.417314,-0.29067,-0.271988,1.301134,-0.354616,-0.517731,-0.399062
59996,-0.489387,-0.490747,-0.656056,1.414869,-0.537373,-0.4147,-0.528264,-0.515470,-0.486193,-0.286964,...,-0.505382,0.678101,-0.349419,-0.417314,-0.29067,-0.271988,-1.103932,-0.354616,-0.517731,-0.399062


In [18]:
allVars = pd.Series(guideMatrix.columns) 
allVarsInteract = pd.Series([ x  for x in selKOs if (x != "GENE_CONTROL_")]) + "*subCellType"

myForm = allVars.append(allVarsInteract)


myFormula = "+".join(myForm)
my_formula = "y~" + myFormula
my_formula

'y~GENE_Vhl_+GENE_Rrp9_+GENE_Cul3_+GENE_Naca_+GENE_Paf1_+GENE_Rnf8_+GENE_Wdr3_+GENE_Rack1_+GENE_Myc_+GENE_Wdr74_+GENE_Dcaf13_+GENE_Mdm4_+GENE_Utp15_+GENE_Nle1_+GENE_Cirh1a_+GENE_Keap1_+GENE_Wdr43_+GENE_Aamp_+GENE_Eif3f_+GENE_Mtor_+GENE_Taf5_+GENE_Prpf4_+GENE_Wdr82_+GENE_Tbl3_+GENE_Bop1_+GENE_Pak1ip1_+GENE_Eif3i_+GENE_Lrr1_+GENE_Prpf19_+GENE_Utp18_+GENE_Kdm2a_+GENE_Wdr75_+GENE_Copb2_+GENE_Thoc3_+GENE_Rbx1_+GENE_Raf1_+GENE_Gemin5_+GENE_Gtf3c1_+GENE_Rnf216_+GENE_Katnb1_+GENE_Zbtb7a_+GENE_Foxo3_+GENE_Spop_+GENE_Pik3r4_+GENE_Wdfy3_+GENE_Klhl6_+GENE_Pja1_+GENE_Ube2f_+GENE_Dido1_+GENE_Ttc3_+GENE_Tceb2_+GENE_Ube2d3_+GENE_Wdr1_+GENE_Traf2_+GENE_Gtf2h2_+GENE_Ube2n_+GENE_Vprbp_+GENE_Bmi1_+GENE_Plrg1_+GENE_Upf1_+GENE_Smu1_+GENE_Crebbp_+GENE_Fbxw7_+GENE_Nsd1_+GENE_Wdfy2_+GENE_Maea_+GENE_Trip12_+GENE_Cebpb_+GENE_Ube2h_+GENE_March6_+GENE_Zbtb25_+GENE_Arih2_+GENE_Wdr26_+GENE_Zmiz2_+GENE_Cul5_+GENE_Pten_+GENE_Ambra1_+GENE_Trp53_+GENE_Nf1_+n_genes+mt_frac+subCellType+GENE_Vhl_*subCellType+GENE_Rrp9_*sub

In [19]:
allResp=myAdata.var_names

In [24]:
%%R -i guideMatrix,expressionMatrix,my_formula,allResp  -w 8 -h 4 -u in
library("MASS")
library("broom")

coefDF = data.frame()
guideMatrix$subCellType = factor(guideMatrix$subCellType, levels=c("Other", "DC2"))

for(i in seq(1,3,1)){
      print(i)
    
          
      tryCatch(
            expr = {
                   guideMatrix["y"] = expressionMatrix[,i]
                   myFit <- glm(formula(my_formula), data = guideMatrix, family = gaussian())
                   myDF <- data.frame(tidy(myFit))
                   myDF$respGene = unlist(allResp)[i]
                   coefDF <- rbind(coefDF, myDF)
            },
            error = function(e){ 
                print(e)
            },
            finally = {
                # (Optional)
                # Do this at the end before quitting the tryCatch structure...
            }
      ) 

}

saveRDS(coefDF, paste0("./cellTypeInteractions.rds"))

[1] 1
[1] 2
[1] 3


In [25]:
%%R
coefDF 
#head(guideMatrix)

                            term      estimate    std.error     statistic
1                    (Intercept) -0.2440136529 2.428699e-02 -10.047091463
2                      GENE_Vhl_  0.0721367618 2.040132e-01   0.353588671
3                     GENE_Rrp9_ -0.2218598737 1.601949e-01  -1.384936997
4                     GENE_Cul3_  0.3252617558 2.130624e-01   1.526603124
5                     GENE_Naca_ -0.1248193809 3.011098e-01  -0.414531155
6                     GENE_Paf1_  0.0037488790 4.464609e-01   0.008396880
7                     GENE_Rnf8_ -0.1023176034 2.883080e-01  -0.354889868
8                     GENE_Wdr3_ -0.2670508219 2.292202e-01  -1.165040708
9                    GENE_Rack1_ -0.3889880954 2.579263e-01  -1.508136590
10                     GENE_Myc_  0.2505938804 3.530100e-01   0.709877660
11                   GENE_Wdr74_  0.2625619615 3.328469e-01   0.788836978
12                  GENE_Dcaf13_  0.2095770551 2.497426e-01   0.839172252
13                    GENE_Mdm4_  0.00

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

